In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import mxnet as mx
from mxnet import gluon
import pickle
import json
import random
import inspect
from scipy import stats
from sklearn.metrics import mean_squared_error
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from pathlib import Path
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepstate import DeepStateEstimator
from gluonts.trainer import Trainer
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator, MultivariateEvaluator
from gluonts.distribution.multivariate_gaussian import MultivariateGaussianOutput
from gluonts.model.predictor import Predictor
from gluonts.model.prophet import ProphetPredictor
from gluonts.model.r_forecast import RForecastPredictor
from indycar.model.NaivePredictor import NaivePredictor
from indycar.model.ZeroPredictor import ZeroPredictor
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

#from indycar.model.stint_predictor_fastrun import *
import indycar.model.stint_simulator as stint

INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU


In [96]:
#
# configurataion
#
# model path:  <_dataset_id>/<_task_id>-<trainid>/
#_dataset_id = 'indy2013-2018-nocarid'
stint.init()
stint._dataset_id = 'indy2013-2018-nocarid-context40'
stint._test_event = 'Indy500-2018'
#_test_event = 'Indy500-2019'

stint._feature_mode = stint.FEATURE_STATUS
stint._context_ratio = 0.

stint._task_id = 'timediff'  # rank,laptime, the trained model's task
stint._run_ts = stint.COL_TIMEDIFF   #COL_LAPTIME,COL_RANK
stint._exp_id='timediff2rank'  #rank, laptime, laptim2rank, timediff2rank... 

stint._train_len = 40

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:192: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0


init: load dataset with 7 races, 58 cars


In [97]:
predictor = stint.load_model(2, 'oracle',trainid='2018')

INFO:root:Using GPU


predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)


In [87]:
df = stint.run_simulation(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE)

start pitlap: 28
simulation done: 33
start pitlap: 29
simulation done: 33
start pitlap: 30
simulation done: 33
start pitlap: 31
simulation done: 33
start pitlap: 32
simulation done: 33
start pitlap: 33
simulation done: 33
start pitlap: 34
simulation done: 33
start pitlap: 35
simulation done: 33
start pitlap: 45
simulation done: 31
start pitlap: 49
simulation done: 31
start pitlap: 51
simulation done: 31
start pitlap: 52
simulation done: 31
start pitlap: 53
simulation done: 31
start pitlap: 59
simulation done: 30
start pitlap: 61
simulation done: 30
start pitlap: 69
simulation done: 29
start pitlap: 88
simulation done: 29
start pitlap: 89
simulation done: 29
start pitlap: 90
simulation done: 29
start pitlap: 91
simulation done: 29
start pitlap: 92
simulation done: 29
start pitlap: 93
simulation done: 29
start pitlap: 94
simulation done: 29
start pitlap: 95
simulation done: 29
start pitlap: 96
simulation done: 29
start pitlap: 97
simulation done: 29
start pitlap: 100
simulation done: 29


In [60]:
%debug

> /N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py(1888)sim_onestep()
   1886                     forecast_laptime_mean = np.mean(forecasts[0].samples, axis=0).reshape((prediction_length))
   1887                     #update target_val
-> 1888                     target_val[endpos-prediction_length:endpos] = forecast_laptime_mean
   1889                     rec[COL_TRACKSTATUS, endpos-prediction_length:endpos] = track_rec[-prediction_length:]
   1890                     rec[COL_LAPSTATUS, endpos-prediction_length:endpos] = lap_rec[-prediction_length:]

ipdb> p forecast_laptime_mean
array([51.39926 , 51.901764], dtype=float32)
ipdb> p endpos
48
ipdb> p target_val
array([ 5.2728  ,  7.0274  ,  8.3547  , 10.0921  , 10.6223  , 11.1947  ,
       12.174   , 13.524   , 14.1075  , 15.0138  , 15.1578  , 16.0177  ,
       16.4355  , 16.7938  , 17.2306  , 17.9794  , 18.6498  , 19.5806  ,
       20.7416  , 21.9129  , 23.0156  , 23.7758  , 24.7464  , 27.1151  ,
       28.6436 

In [88]:
df

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
0,13,28,11.0,12.0,1.0,1,11.0,0.0,0
1,14,29,5.0,1.0,-4.0,-1,6.0,1.0,1
2,59,29,27.0,28.0,1.0,1,22.0,-5.0,-1
3,3,30,7.0,4.0,-3.0,-1,7.0,0.0,0
4,20,30,0.0,0.0,0.0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...
146,25,156,18.0,0.0,-18.0,-1,14.0,-4.0,-1
147,64,156,6.0,3.0,-3.0,-1,5.0,-1.0,-1
148,59,157,21.0,19.0,-2.0,-1,19.0,-2.0,-1
149,60,157,16.0,1.0,-15.0,-1,4.0,-12.0,-1


### test

In [ ]:
allpits, pitmat, maxlap = stint.get_pitlaps(verbose=True)
#print(len(allpits), maxlap)
nextpit, maxnext = stint.get_nextpit(pitmat, 30)

print(nextpit, maxnext)

In [ ]:
len(pitmat)

In [ ]:
pitmat

In [89]:
df[df['carno']==12]

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
11,12,31,3.0,5.0,2.0,1,2.0,-1.0,-1
38,12,49,5.0,0.0,-5.0,-1,0.0,-5.0,-1
86,12,93,0.0,0.0,0.0,0,0.0,0.0,0
117,12,128,0.0,0.0,0.0,0,0.0,0.0,0


In [90]:
correct = df[df['sign']==df['pred_sign']] 
acc = len(correct)/len(df)
acc

0.6291390728476821

In [94]:
stint.get_evalret(df)

pred: acc=0.6291390728476821, mae=3.4701986754966887,3.4701986754966887, rmse=27.576158940397352,r2=-0.5046232483344819, acc_naive=0.1456953642384106, mae_naive=4.529801324503311


(0.6291390728476821,
 3.4701986754966887,
 27.576158940397352,
 -0.5046232483344819)

In [98]:
ret ={ }
ret['oracle'] = df
ret['curtrack'] = stint.run_simulation(predictor, 2, stint.freq, datamode=stint.MODE_TESTCURTRACK )

start pitlap: 28
simulation done: 33
start pitlap: 29
simulation done: 33
start pitlap: 30
simulation done: 33
start pitlap: 31
simulation done: 33
start pitlap: 32
simulation done: 33
start pitlap: 33
simulation done: 33
start pitlap: 34
simulation done: 33
start pitlap: 35
simulation done: 33
start pitlap: 45
simulation done: 31
start pitlap: 49
simulation done: 31
start pitlap: 51
simulation done: 31
start pitlap: 52
simulation done: 31
start pitlap: 53
simulation done: 31
start pitlap: 59
simulation done: 30
start pitlap: 61
simulation done: 30
start pitlap: 69
simulation done: 29
start pitlap: 88
simulation done: 29
start pitlap: 89
simulation done: 29
start pitlap: 90
simulation done: 29
start pitlap: 91
simulation done: 29
start pitlap: 92
simulation done: 29
start pitlap: 93
simulation done: 29
start pitlap: 94
simulation done: 29
start pitlap: 95
simulation done: 29
start pitlap: 96
simulation done: 29
start pitlap: 97
simulation done: 29
start pitlap: 100
simulation done: 29


In [99]:
stint.get_evalret(ret['curtrack'])

pred: acc=0.6026490066225165, mae=5.3841059602649,5.3841059602649, rmse=61.609271523178805,r2=-0.8693351182745332, acc_naive=0.1456953642384106, mae_naive=4.529801324503311


(0.6026490066225165, 5.3841059602649, 61.609271523178805, -0.8693351182745332)

In [101]:
ret['zerotrack'] = stint.run_simulation(predictor, 2, stint.freq, datamode=stint.MODE_TESTZERO )

start pitlap: 28
simulation done: 33
start pitlap: 29
simulation done: 33
start pitlap: 30
simulation done: 33
start pitlap: 31
simulation done: 33
start pitlap: 32
simulation done: 33
start pitlap: 33
simulation done: 33
start pitlap: 34
simulation done: 33
start pitlap: 35
simulation done: 33
start pitlap: 45
simulation done: 31
start pitlap: 49
simulation done: 31
start pitlap: 51
simulation done: 31
start pitlap: 52
simulation done: 31
start pitlap: 53
simulation done: 31
start pitlap: 59
simulation done: 30
start pitlap: 61
simulation done: 30
start pitlap: 69
simulation done: 29
start pitlap: 88
simulation done: 29
start pitlap: 89
simulation done: 29
start pitlap: 90
simulation done: 29
start pitlap: 91
simulation done: 29
start pitlap: 92
simulation done: 29
start pitlap: 93
simulation done: 29
start pitlap: 94
simulation done: 29
start pitlap: 95
simulation done: 29
start pitlap: 96
simulation done: 29
start pitlap: 97
simulation done: 29
start pitlap: 100
simulation done: 29


In [102]:
stint.get_evalret(ret['zerotrack'])

pred: acc=0.609271523178808, mae=6.006622516556291,6.006622516556291, rmse=71.2251655629139,r2=-0.6695263042106063, acc_naive=0.1456953642384106, mae_naive=4.529801324503311


(0.609271523178808, 6.006622516556291, 71.2251655629139, -0.6695263042106063)

### test sim by pred

In [140]:
#
# configurataion
#
# model path:  <_dataset_id>/<_task_id>-<trainid>/
#_dataset_id = 'indy2013-2018-nocarid'
stint.init()
stint._dataset_id = 'indy2013-2018-nocarid-context40'
stint._test_event = 'Indy500-2018'
#_test_event = 'Indy500-2019'

stint._feature_mode = stint.FEATURE_STATUS
stint._context_ratio = 0.

stint._task_id = 'timediff'  # rank,laptime, the trained model's task
stint._run_ts = stint.COL_TIMEDIFF   #COL_LAPTIME,COL_RANK
stint._exp_id='timediff2rank'  #rank, laptime, laptim2rank, timediff2rank... 

stint._train_len = 40
predictor = stint.load_model(2, 'oracle',trainid='2018')

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)


In [141]:
df = stint.run_simulation_pred(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE)

start pitlap: 28
update lapstatus done.
simulation done: 33
start pitlap: 29
update lapstatus done.
simulation done: 33
start pitlap: 30
update lapstatus done.
simulation done: 33
start pitlap: 31
update lapstatus done.
simulation done: 33
start pitlap: 32
update lapstatus done.
simulation done: 33
start pitlap: 33
update lapstatus done.
simulation done: 33
start pitlap: 34
update lapstatus done.
simulation done: 33
start pitlap: 35
update lapstatus done.
simulation done: 33
start pitlap: 45
update lapstatus done.
simulation done: 31
start pitlap: 49
update lapstatus done.
simulation done: 31
start pitlap: 51
update lapstatus done.
simulation done: 31
start pitlap: 52
update lapstatus done.
simulation done: 31
start pitlap: 53
update lapstatus done.
simulation done: 31
start pitlap: 59
update lapstatus done.
simulation done: 30
start pitlap: 61
update lapstatus done.
simulation done: 30
start pitlap: 69
update lapstatus done.
simulation done: 29
start pitlap: 88
update lapstatus done.


In [137]:
%debug

> /N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py(1800)update_onets()
   1798     while True:
   1799         caution_laps_instint = int(rec[COL_CAUTION_LAPS_INSTINT, curpos])
-> 1800         laps_instint = int(rec[COL_LAPS_INSTINT, curpos])
   1801 
   1802         if caution_laps_instint>10:

ipdb> p rec[COL_LAPSTATUS,curpos]
0.0
ipdb> quit


In [142]:
stint.get_evalret(df)

pred: acc=0.6490066225165563, mae=2.880794701986755,2.880794701986755, rmse=16.629139072847682,r2=0.32663271068867405, acc_naive=0.152317880794702, mae_naive=4.490066225165563


(0.6490066225165563,
 2.880794701986755,
 16.629139072847682,
 0.32663271068867405)

In [144]:
ret = {}
for i in range(10):
    ret[i] = stint.run_simulation_pred(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE)


start pitlap: 28
update lapstatus done.
simulation done: 33
start pitlap: 29
update lapstatus done.
simulation done: 33
start pitlap: 30
update lapstatus done.
simulation done: 33
start pitlap: 31
update lapstatus done.
simulation done: 33
start pitlap: 32
update lapstatus done.
simulation done: 33
start pitlap: 33
update lapstatus done.
simulation done: 33
start pitlap: 34
update lapstatus done.
simulation done: 33
start pitlap: 35
update lapstatus done.
simulation done: 33
start pitlap: 45
update lapstatus done.
simulation done: 31
start pitlap: 49
update lapstatus done.
simulation done: 31
start pitlap: 51
update lapstatus done.
simulation done: 31
start pitlap: 52
update lapstatus done.
simulation done: 31
start pitlap: 53
update lapstatus done.
simulation done: 31
start pitlap: 59
update lapstatus done.
simulation done: 30
start pitlap: 61
update lapstatus done.
simulation done: 30
start pitlap: 69
update lapstatus done.
simulation done: 29
start pitlap: 88
update lapstatus done.


simulation done: 25
start pitlap: 169
update lapstatus done.
simulation done: 0
start pitlap: 170
update lapstatus done.
simulation done: 0
start pitlap: 171
update lapstatus done.
simulation done: 0
start pitlap: 172
update lapstatus done.
simulation done: 0
start pitlap: 173
update lapstatus done.
simulation done: 0
start pitlap: 174
update lapstatus done.
simulation done: 0
start pitlap: 175
update lapstatus done.
simulation done: 0
start pitlap: 176
update lapstatus done.
simulation done: 0
start pitlap: 179
update lapstatus done.
simulation done: 0
start pitlap: 184
update lapstatus done.
simulation done: 0
start pitlap: 186
update lapstatus done.
simulation done: 0
start pitlap: 189
update lapstatus done.
simulation done: 0
start pitlap: 190
update lapstatus done.
simulation done: 0
start pitlap: 194
update lapstatus done.
simulation done: 0
start pitlap: 195
update lapstatus done.
simulation done: 0
start pitlap: 196
update lapstatus done.
simulation done: 0
start pitlap: 28
upd

simulation done: 28
start pitlap: 134
update lapstatus done.
simulation done: 28
start pitlap: 135
update lapstatus done.
simulation done: 27
start pitlap: 136
update lapstatus done.
simulation done: 27
start pitlap: 137
update lapstatus done.
simulation done: 27
start pitlap: 139
update lapstatus done.
simulation done: 27
start pitlap: 140
update lapstatus done.
simulation done: 27
start pitlap: 141
update lapstatus done.
simulation done: 27
start pitlap: 142
update lapstatus done.
simulation done: 27
start pitlap: 145
update lapstatus done.
simulation done: 26
start pitlap: 149
update lapstatus done.
simulation done: 26
start pitlap: 151
update lapstatus done.
simulation done: 26
start pitlap: 155
update lapstatus done.
simulation done: 25
start pitlap: 156
update lapstatus done.
simulation done: 25
start pitlap: 157
update lapstatus done.
simulation done: 25
start pitlap: 159
update lapstatus done.
simulation done: 25
start pitlap: 167
update lapstatus done.
simulation done: 25
star

simulation done: 29
start pitlap: 103
update lapstatus done.
simulation done: 29
start pitlap: 105
update lapstatus done.
simulation done: 29
start pitlap: 106
update lapstatus done.
simulation done: 29
start pitlap: 107
update lapstatus done.
simulation done: 29
start pitlap: 109
update lapstatus done.
simulation done: 28
start pitlap: 119
update lapstatus done.
simulation done: 28
start pitlap: 120
update lapstatus done.
simulation done: 28
start pitlap: 123
update lapstatus done.
simulation done: 28
start pitlap: 124
update lapstatus done.
simulation done: 28
start pitlap: 126
update lapstatus done.
simulation done: 28
start pitlap: 127
update lapstatus done.
simulation done: 28
start pitlap: 128
update lapstatus done.
simulation done: 28
start pitlap: 129
update lapstatus done.
simulation done: 28
start pitlap: 130
update lapstatus done.
simulation done: 28
start pitlap: 132
update lapstatus done.
simulation done: 28
start pitlap: 133
update lapstatus done.
simulation done: 28
star

simulation done: 31
start pitlap: 53
update lapstatus done.
simulation done: 31
start pitlap: 59
update lapstatus done.
simulation done: 30
start pitlap: 61
update lapstatus done.
simulation done: 30
start pitlap: 69
update lapstatus done.
simulation done: 29
start pitlap: 88
update lapstatus done.
simulation done: 29
start pitlap: 89
update lapstatus done.
simulation done: 29
start pitlap: 90
update lapstatus done.
simulation done: 29
start pitlap: 91
update lapstatus done.
simulation done: 29
start pitlap: 92
update lapstatus done.
simulation done: 29
start pitlap: 93
update lapstatus done.
simulation done: 29
start pitlap: 94
update lapstatus done.
simulation done: 29
start pitlap: 95
update lapstatus done.
simulation done: 29
start pitlap: 96
update lapstatus done.
simulation done: 29
start pitlap: 97
update lapstatus done.
simulation done: 29
start pitlap: 100
update lapstatus done.
simulation done: 29
start pitlap: 102
update lapstatus done.
simulation done: 29
start pitlap: 103


simulation done: 33
start pitlap: 29
update lapstatus done.
simulation done: 33
start pitlap: 30
update lapstatus done.
simulation done: 33
start pitlap: 31
update lapstatus done.
simulation done: 33
start pitlap: 32
update lapstatus done.
simulation done: 33
start pitlap: 33
update lapstatus done.
simulation done: 33
start pitlap: 34
update lapstatus done.
simulation done: 33
start pitlap: 35
update lapstatus done.
simulation done: 33
start pitlap: 45
update lapstatus done.
simulation done: 31
start pitlap: 49
update lapstatus done.
simulation done: 31
start pitlap: 51
update lapstatus done.
simulation done: 31
start pitlap: 52
update lapstatus done.
simulation done: 31
start pitlap: 53
update lapstatus done.
simulation done: 31
start pitlap: 59
update lapstatus done.
simulation done: 30
start pitlap: 61
update lapstatus done.
simulation done: 30
start pitlap: 69
update lapstatus done.
simulation done: 29
start pitlap: 88
update lapstatus done.
simulation done: 29
start pitlap: 89
upd

In [148]:
acc = []
for i in ret.keys():
    df = ret[i]
    _x = stint.get_evalret(df)
    acc.append(_x)

b = np.array(acc)
np.mean(b, axis=0)

pred: acc=0.6688741721854304, mae=2.880794701986755,2.880794701986755, rmse=18.165562913907284,r2=0.14685223939623637, acc_naive=0.152317880794702, mae_naive=4.490066225165563
pred: acc=0.6556291390728477, mae=2.801324503311258,2.801324503311258, rmse=16.788079470198674,r2=0.22749912212369383, acc_naive=0.1456953642384106, mae_naive=4.516556291390729
pred: acc=0.6887417218543046, mae=2.814569536423841,2.814569536423841, rmse=17.70198675496689,r2=0.21604072626696613, acc_naive=0.16556291390728478, mae_naive=4.483443708609271
pred: acc=0.6556291390728477, mae=2.9403973509933774,2.9403973509933774, rmse=18.344370860927153,r2=0.24874272577052947, acc_naive=0.15894039735099338, mae_naive=4.496688741721854
pred: acc=0.6688741721854304, mae=2.9735099337748343,2.9735099337748343, rmse=19.370860927152318,r2=0.09289472711365232, acc_naive=0.152317880794702, mae_naive=4.503311258278146
pred: acc=0.6556291390728477, mae=2.8410596026490067,2.8410596026490067, rmse=17.56953642384106,r2=0.24227527558

array([ 0.67086093,  2.90397351, 18.65364238,  0.19329563])

In [145]:
a = [(1,2,3),(2,3,4)]
np.array(a)

array([[1, 2, 3],
       [2, 3, 4]])

In [147]:
len(ret)

10